# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excels\Only_Male\GeneCpG\Apathy
df = pd.read_csv('ApathyCateg_Methylome_Male.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\Only_Male\GeneCpG\Apathy
(308, 3572)


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,MDS_Apathy_Category,cg19958696,cg05962325,...,cg19795267,cg16922840,cg19628277,cg26480988,cg01673347,cg12994818,cg09008531,cg16463148,cg04021430,cg04863888
0,200991620021_R01C01,3001,1,1,2,65.1,4,0,0.014601,0.068728,...,0.630319,0.851560,0.521620,0.495925,0.830013,0.076462,0.728590,0.246918,0.039718,0.024071
1,200989060135_R07C01,3004,1,2,0,59.4,3,0,0.013334,0.038642,...,0.655485,0.867540,0.546537,0.511772,0.802469,0.033789,0.634950,0.193630,0.023717,0.023474
2,200973410167_R04C01,3007,1,1,2,64.5,3,0,0.015753,0.079013,...,0.719083,0.901237,0.545145,0.540302,0.669380,0.036798,0.766937,0.148859,0.020356,0.023666


In [4]:
df1 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender', 'Age', 'AgeCate','MDS_Apathy_Category'], axis=1)
df1.head(2)

,APPRDX,cg19958696,cg05962325,cg15093662,cg04873221,cg05903298,cg19020436,cg14994056,cg05973813,cg18337422,...,cg19795267,cg16922840,cg19628277,cg26480988,cg01673347,cg12994818,cg09008531,cg16463148,cg04021430,cg04863888
0,1,0.014601,0.068728,0.019329,0.021747,0.012923,0.017399,0.033939,0.020282,0.017160,...,0.630319,0.85156,0.521620,0.495925,0.830013,0.076462,0.72859,0.246918,0.039718,0.024071
1,2,0.013334,0.038642,0.017604,0.021562,0.012244,0.016683,0.036065,0.021018,0.013908,...,0.655485,0.86754,0.546537,0.511772,0.802469,0.033789,0.63495,0.193630,0.023717,0.023474


In [5]:
df1 = df1.transpose()
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,298,299,300,301,302,303,304,305,306,307
APPRDX,1.000000,2.000000,1.000000,1.00000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,...,1.00000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
cg19958696,0.014601,0.013334,0.015753,0.01512,0.013551,0.012227,0.013772,0.010181,0.011954,0.012183,...,0.01305,0.011279,0.007166,0.013514,0.010569,0.012322,0.014182,0.01322,0.013479,0.012706


In [6]:
df1.rename(columns=df1.iloc[0], inplace = True)


In [7]:
df1.index.name ='CpG'
df1.reset_index(inplace=True)
df1.head(2)

,CpG,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
0,APPRDX,1.000000,2.000000,1.000000,1.00000,2.000000,1.000000,2.000000,1.000000,1.000000,...,1.00000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
1,cg19958696,0.014601,0.013334,0.015753,0.01512,0.013551,0.012227,0.013772,0.010181,0.011954,...,0.01305,0.011279,0.007166,0.013514,0.010569,0.012322,0.014182,0.01322,0.013479,0.012706


In [8]:
df1 = df1.drop ([0], axis = 0)
df1.head(2)

,CpG,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
1,cg19958696,0.014601,0.013334,0.015753,0.015120,0.013551,0.012227,0.013772,0.010181,0.011954,...,0.013050,0.011279,0.007166,0.013514,0.010569,0.012322,0.014182,0.013220,0.013479,0.012706
2,cg05962325,0.068728,0.038642,0.079013,0.096791,0.095520,0.054082,0.055584,0.035805,0.050055,...,0.042397,0.052875,0.049444,0.065591,0.039504,0.027668,0.045231,0.053975,0.045178,0.053672


In [9]:
df2 = pd.read_csv ("Apathy_BorutaMale_Top50.csv")
df2.head(2)

,CpG
0,cg13959611
1,cg07897734


In [10]:
df3 = pd.merge (df1, df2, on= "CpG", how= "inner")
df3.head(2)

,CpG,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
0,cg01231141,0.482556,0.360877,0.487274,0.459429,0.465203,0.606473,0.436593,0.373917,0.327839,...,0.829957,0.488308,0.602331,0.365045,0.288947,0.363133,0.487670,0.754874,0.504958,0.535361
1,cg03993109,0.916928,0.935908,0.913368,0.926767,0.874346,0.939532,0.894196,0.722396,0.956740,...,0.951789,0.846448,0.948922,0.846323,0.900594,0.942222,0.890956,0.959220,0.942665,0.955105


In [11]:
df3 = df3.transpose()
df3.head(2)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
CpG,cg01231141,cg03993109,cg07584368,cg24073995,cg12787030,cg10957063,cg10400309,cg03984780,cg11865360,cg18060171,...,cg04286737,cg23362473,cg16643803,cg01447828,cg05124260,cg26126367,cg07897734,cg06552618,cg00538060,cg13959611
1.0,0.482556,0.916928,0.868743,0.638848,0.722242,0.630408,0.013044,0.065929,0.279839,0.583955,...,0.940614,0.917361,0.029913,0.441146,0.533905,0.82569,0.842938,0.035788,0.785801,0.044262


In [12]:
df3.rename(columns=df3.iloc[0], inplace = True)


In [13]:
df3.index.name ='APPRDX'
df3.reset_index(inplace=True)
df3.head(2)

,APPRDX,cg01231141,cg03993109,cg07584368,cg24073995,cg12787030,cg10957063,cg10400309,cg03984780,cg11865360,...,cg04286737,cg23362473,cg16643803,cg01447828,cg05124260,cg26126367,cg07897734,cg06552618,cg00538060,cg13959611
0,CpG,cg01231141,cg03993109,cg07584368,cg24073995,cg12787030,cg10957063,cg10400309,cg03984780,cg11865360,...,cg04286737,cg23362473,cg16643803,cg01447828,cg05124260,cg26126367,cg07897734,cg06552618,cg00538060,cg13959611
1,1.0,0.482556,0.916928,0.868743,0.638848,0.722242,0.630408,0.013044,0.065929,0.279839,...,0.940614,0.917361,0.029913,0.441146,0.533905,0.82569,0.842938,0.035788,0.785801,0.044262


In [14]:
df3 = df3.drop ([0], axis = 0)
df3.head(2)

,APPRDX,cg01231141,cg03993109,cg07584368,cg24073995,cg12787030,cg10957063,cg10400309,cg03984780,cg11865360,...,cg04286737,cg23362473,cg16643803,cg01447828,cg05124260,cg26126367,cg07897734,cg06552618,cg00538060,cg13959611
1,1.0,0.482556,0.916928,0.868743,0.638848,0.722242,0.630408,0.013044,0.065929,0.279839,...,0.940614,0.917361,0.029913,0.441146,0.533905,0.82569,0.842938,0.035788,0.785801,0.044262
2,2.0,0.360877,0.935908,0.864397,0.579817,0.784267,0.632613,0.010265,0.066287,0.098743,...,0.95365,0.900393,0.033853,0.559728,0.684175,0.852642,0.894486,0.031247,0.835675,0.048167


In [15]:
df3.reset_index(inplace = True, drop = True)
df3.head(3)

,APPRDX,cg01231141,cg03993109,cg07584368,cg24073995,cg12787030,cg10957063,cg10400309,cg03984780,cg11865360,...,cg04286737,cg23362473,cg16643803,cg01447828,cg05124260,cg26126367,cg07897734,cg06552618,cg00538060,cg13959611
0,1.0,0.482556,0.916928,0.868743,0.638848,0.722242,0.630408,0.013044,0.065929,0.279839,...,0.940614,0.917361,0.029913,0.441146,0.533905,0.82569,0.842938,0.035788,0.785801,0.044262
1,2.0,0.360877,0.935908,0.864397,0.579817,0.784267,0.632613,0.010265,0.066287,0.098743,...,0.95365,0.900393,0.033853,0.559728,0.684175,0.852642,0.894486,0.031247,0.835675,0.048167
2,1.0,0.487274,0.913368,0.910577,0.668722,0.849996,0.69744,0.013645,0.051893,0.137535,...,0.944169,0.932135,0.025803,0.444385,0.704398,0.901398,0.923446,0.041458,0.818536,0.033433


In [16]:
df3.shape

(308, 51)

In [17]:
df3 ['APPRDX'] = df3 ['APPRDX'].apply(int)

## RanFor 1000 times, CpG only 

In [18]:
df3.head(2)

,APPRDX,cg01231141,cg03993109,cg07584368,cg24073995,cg12787030,cg10957063,cg10400309,cg03984780,cg11865360,...,cg04286737,cg23362473,cg16643803,cg01447828,cg05124260,cg26126367,cg07897734,cg06552618,cg00538060,cg13959611
0,1,0.482556,0.916928,0.868743,0.638848,0.722242,0.630408,0.013044,0.065929,0.279839,...,0.940614,0.917361,0.029913,0.441146,0.533905,0.82569,0.842938,0.035788,0.785801,0.044262
1,2,0.360877,0.935908,0.864397,0.579817,0.784267,0.632613,0.010265,0.066287,0.098743,...,0.95365,0.900393,0.033853,0.559728,0.684175,0.852642,0.894486,0.031247,0.835675,0.048167


In [19]:
df3.shape

(308, 51)

In [20]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(90))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [21]:
dframe1.head(2)

,Accuracy,Std_Dev
0,0.617778,0.091382
1,0.633333,0.104822


In [22]:
dframe1.to_csv ("Apathy_BorutaCpG_Male_1000RF.csv")